# 8.3 Datenkapselung, Datenabstraktion und Geheimnisprinzip

<img src="IMG/logo.png" />
<a href="0_Einfuehrung.ipynb">&larr; Einführung/Inhalt</a>

Unter dem Begriff Datenkapselung versteht man den Schutz von Daten bzw. Attributen vor dem unmittelbaren Zugriff. Dies wird anhand von sogenannten Zugriffsmethoden bewerkstelligt. Anhand dieser Zugriffsmethoden wird definiert, wie auf die Daten eines Objekt zugegriffen werden kann. **Ein direkter Zugriff auf die Attribute sollte vermieden werden!** Die Zugriffsmethoden erlauben dem Entwickler einer Klasse den Zugriff auf die Attribute zu steuern und ggf. weitere Restriktionen innerhalb einer Änderungsmethode zu implementieren (z.B. Wertebereiche auszugrenzen). Solle Zugriffsmethoden werden mit *getter* (Abfragemethode) und *setter* (Änderungsmethode) bezeichnet.

Durch die oben erwähnten Zugriffsmethoden wird gleichzeitig ein invariantes Interface geschaffen (Abstraktion), d.h. dem Benutzer wird nur die Schnittstelle für die Interaktion "angeboten" und die Implementierungsdetails bleiben verborgen. Somit können jederzeit Änderungen an der Implementierung vorgenommen werden, ohne dass sich die Benutzersicht (Interface) ändert.

Damit das Beispiel nicht unübersichtlich wird, wird das oben Erwähnte nur für das Attribut *port* gezeigt (dasselbe könnte für weitere Attribute analog gemacht werden).

In [18]:
class RGB:
    def __init__(self, port='COM5', default_color=(0, 0, 0)):
        self.port = port
        self.default_color = default_color

    def get_port(self):  # Abfragemethode (getter)
        return self.port

    def set_port(self, port):  # Änderungsmethode (setter)
        # Hier können weitere Restriktionen, wie z.B. Ausgrenzung Wertebereich durchgeführt werden.
        self.port = port

In [19]:
rgb = RGB()
rgb.get_port()  # Abfragen des aktuellen Ports

'COM5'

In [20]:
rgb.set_port('COM6')  # Ändern des Ports
rgb.get_port()

'COM6'

Das obige Beispiel mit Abfragen und Anpassung der Port-Bezeichnung zeigt die Idee, wie nun mit diesen Zugriffsmethoden gearbeitet werden soll. Unschön ist in der aktuellen Umsetzung der Klasse, der Umstand, dass weiterhin ein direkter Zugriff auf das Attribut selbst möglich ist. Dies sollte gemäss obiger Prämisse programmatisch unterbunden werden.

In [21]:
rgb.port = 'COM7'  # unschöner, direkter Zugriff auf das Attribut port!
rgb.get_port()

'COM7'

## 8.3.1 Öffentliche, geschützte und private Attribute/Methoden

Obiges Beispiel (direkter Zugriff auf das Attribut *port*) zeigt, dass die, für die Port-Änderung zuständige *set_port*-Methode, umgangen werden kann. Damit dies verhindert wird, existieren Möglichkeiten, um die Zugriffsmöglichkeiten (lesen und/oder schreiben) von Attributen/Methoden zu definieren. Python kennt die folgenden drei Definitionen:
- **Public**, z. B. `port` (lesen und schreiben möglich und auch gewollt)
- **Protected**: z. B. `_port` (lesen und schreiben möglich, sollte aber nicht gemacht werden, siehe Vererbung)
- **Private**: z. B. `__port` (von aussen nicht sichtbar und nicht benutzbar)

Wie diese Zugriffsmöglichkeiten verwendet werden, zeigt das nächste Beispiel, die überarbeitete RGB-Klasse.

In [22]:
class RGB:
    def __init__(self, port='COM5', default_color=(0, 0, 0)):
        self.set_port(port)
        ...

    def get_port(self):  # Abfragemethode (getter)
        return self.__port

    def set_port(self, port):  # Änderungsmethode (setter)
        self.__port = port  # Attribut port ist private definiert

In [23]:
rgb = RGB()
rgb.get_port()  # nur noch Zugriff via getter-Methode möglich (rgb.__port nicht möglich)

'COM5'

Analog zur Datenkapselung auf Ebene Attributen, wird diese auch auf der Ebene Methoden angewendet. D.h. Methoden können auch *public*, *protected* und *private* deklariert werden. Im aktuellen Beispiel sind sämtliche Zugriffsmethoden frei (public) zugänglich. <br>
Für eine vollständige Umsetzung des invarianten Interfaces müssen die Methoden zwingend, analog der Attribute, gekapselt werden.

In [24]:
class RGB:
    def __init__(self, port='COM5', default_color=(0, 0, 0)):
        self.__set_port(port)
        ...

    def __get_port(self):  # Abfragemethode (getter)
        return self.__port

    def __set_port(self, port):  # Änderungsmethode (setter)
        self.__port = port  # Attribut port ist private definiert

In dieser neuen Klasse RGB sind nun sämtliche Zugriffsmethoden *private* deklariert, d.h. der Benutzer hat somit keine Möglichkeit mehr auf die beiden Attribute *port* und *default_color* zuzugreifen. Dieser Zustand macht aber sicherlich keinen Sinn. <br>

## 8.3.2 Property

Mittels den sogenannten *Properties* kann der Zugriff für den Benutzer auf eine sehr komfortable Art und Weise gesteuert werden. Die Schnittstelle ist somit komplett entkoppelt und somit braucht der Benutzer auch keinerlei Kenntnisse über die Namen der Zugriffsmethoden. Es genügt den entsprechenden Attribut-Namen zu kennen. Für den Benutzer sieht es denn so aus, als ob er direkt auf das entsprechende Attribut zugreifen würde.

In [25]:
class RGB:
    def __init__(self, port='COM5', default_color=(0, 0, 0)):
        self.__set_port(port)
        ...

    def __get_port(self):  # Abfragemethode (getter)
        return self.__port

    def __set_port(self, port):  # Änderungsmethode (setter)
        self.__port = port  # Attribut port ist private definiert

    port = property(__get_port, __set_port)  # Übergabe der Funktion und kein Funktionsaufruf!

In [26]:
rgb = RGB()
rgb.port  # Attriburt kann nun wieder "scheinbar" direkt abgerufen werden, im Hintergrund wird aber getter-Methode verwendet

'COM5'

In [27]:
rgb.port = 'COM6'  # Attribut zuweisen ("scheinbar" direkt, aber im Hintergrund wird setter-Methode verwendet)
rgb.port

'COM6'

Mit *decorators* kann das obige noch kompakter/eleganter geschrieben werden.

In [28]:
class RGB:
    def __init__(self, port='COM5', default_color=(0, 0, 0)):
        self.__port = port
        ...

    @property
    def port(self):
        return self.__port

    @port.setter
    def port(self, port):
        self.__port = port

In [29]:
rgb = RGB()
rgb.port

'COM5'

In [30]:
rgb.port = 'COM6'
rgb.port

'COM6'

---
<p style='text-align: right; font-size: 70%;'>Grundlagen Python (PYT_G01) / 2024</p>